In [1]:
import sys; sys.path.append('..')
import torch
from dual_attn_blocks import DualAttnEncoderBlock, DualAttnDecoderBlock
from symbol_retrieval import PositionRelativeSymbolRetriever, SymbolicAttention

Documentation

In [2]:
DualAttnEncoderBlock?

Init signature:
DualAttnEncoderBlock(
    d_model: int,
    n_heads_sa: int,
    n_heads_ra: int,
    dff: int,
    activation: str,
    dropout_rate: float,
    norm_first: bool,
    norm_type: str = 'layernorm',
    sa_kwargs: dict = None,
    ra_kwargs: dict = None,
    ra_type: str = 'relational_attention',
    bias: bool = True,
    causal: bool = False,
)
Docstring:     
Base class for all neural network modules.

Your models should also subclass this class.

Modules can also contain other Modules, allowing to nest them in
a tree structure. You can assign the submodules as regular attributes::

    import torch.nn as nn
    import torch.nn.functional as F

    class Model(nn.Module):
        def __init__(self):
            super().__init__()
            self.conv1 = nn.Conv2d(1, 20, 5)
            self.conv2 = nn.Conv2d(20, 20, 5)

        def forward(self, x):
            x = F.relu(self.conv1(x))
            return F.relu(self.conv2(x))

Submodules assigned in this way will be 

Dual-Attention Encoder Block with Symbolic Attention

In [3]:
d, l = 64, 10

symbol_retriever = SymbolicAttention(d_model=d, n_heads=4, n_symbols=10)
da_enc_block = DualAttnEncoderBlock(d_model=d, n_heads_sa=4, n_heads_ra=4, dff=d*4, activation='swiglu',
    dropout_rate=0., norm_first=True, ra_kwargs=dict(symmetric_rels=True, use_relative_positional_symbols=False))

In [4]:
x = torch.randn(1, 10, d)
s = symbol_retriever(x)
y, *_ = da_enc_block(x, s)
print(f'x: {x.shape}, s: {s.shape}, y: {y.shape}')

x: torch.Size([1, 10, 64]), s: torch.Size([1, 10, 64]), y: torch.Size([10, 64])


Dual-Attention Encoder Block with Position-Relative Symbols

In [5]:
d, l = 64, 10

symbol_retriever = PositionRelativeSymbolRetriever(symbol_dim=d, max_rel_pos=l)
da_enc_block = DualAttnEncoderBlock(d_model=d, n_heads_sa=4, n_heads_ra=4, dff=d*4, activation='swiglu',
    dropout_rate=0., norm_first=True, ra_kwargs=dict(symmetric_rels=True, use_relative_positional_symbols=True))

In [6]:
s.shape

torch.Size([1, 10, 64])

In [7]:
x = torch.randn(1, 10, d)
s = symbol_retriever(x)
y, *_ = da_enc_block(x, s)
print(f'x: {x.shape}, s: {s.shape}, y: {y.shape}')

x: torch.Size([1, 10, 64]), s: torch.Size([10, 10, 64]), y: torch.Size([10, 64])
